# Machine Learning

In [1]:
import numpy as np
import pandas as pd
import spacy
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import log_loss
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
import warnings


%matplotlib inline
pd.set_option("display.max_columns", None)
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('C:\\Users\\jltsa\\Desktop\\Cust_complaints\\feature_sets\\ml_data1.csv')

In [3]:
df.head()

,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company,submitted_via,timely_response
0,Credit card,None,Billing statement,None,Year end summary provided by citi shows balanc...,"CITIBANK, N.A.",Web,True
1,Debt collection,"Other (i.e. phone, health club, etc.)",Disclosure verification of debt,Not given enough info to verify debt,I received a letter from a debt collector clai...,"Amsher Collection Services, Inc.",Web,True
2,Debt collection,"Other (i.e. phone, health club, etc.)",Cont'd attempts collect debt not owed,Debt is not mine,I HAVE SUBMITTED COMPLAINTS TO OF UPSTATE NY C...,"Credit Protection Association, L.P.",Web,True
3,Debt collection,"Other (i.e. phone, health club, etc.)",Cont'd attempts collect debt not owed,Debt resulted from identity theft,has accounts in my name that were fraudulently...,"Convergent Resources, Inc.",Web,True
4,Credit card,None,Late fee,None,When making online through personal bank payme...,SYNCHRONY FINANCIAL,Web,True


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383840 entries, 0 to 383839
Data columns (total 7 columns):
product                         383840 non-null object
sub_product                     383840 non-null object
issue                           383840 non-null object
sub_issue                       383840 non-null object
consumer_complaint_narrative    383840 non-null object
company                         383840 non-null object
submitted_via                   383840 non-null object
dtypes: object(7)
memory usage: 20.5+ MB


## Measuring Success

In [10]:
df.groupby('timely_response').count()['company'].sort_values(ascending=False)

timely_response
True     372452
False     11388
Name: company, dtype: int64

In [11]:
11388/(372452+11388)

0.029668611921634013

Because the data has only around 3% of the labels calssified as False or not responsign appropriately in time, accuracy will be a bad metric of success.  For instance, the model can classify everything as True and have an accuracy rate of 97%.

Instead, we can use log loss as a measure of success.  It is actually a measure of error.  In this case we want log loss to be as small as possible as opposed to accuracy.

Oen take away is that it isbetter to be less confident than confident and wrong about a prediction.

In [3]:
# Log loss can be calculated with numpy
def compute_log_loss(predicted, actual, eps=1e-14):
    """
    predicted: probabilities as floats between 0 and 1
    actual: binary labels, 0=no or 1=yes
    eps: log(0) is inf, need to offset predicted values by eps from 0 to 1
    """
    predicted = np.clip(predicted, eps, 1- eps)
    loss = -1 * np.mean(actual * np.log(predicted) + (1 - actual)
              * np.log(1 - predicted))
    return loss

In [15]:
#we are saying we are predicting the value is 1 with 90% probability
#But the the actual label is 0
compute_log_loss(predicted=0.9, actual=0)

2.302585092994046

In [33]:
#scikit-learn implementation
log_loss([0,1], [0.9, 0.1])

2.302585092994046

In [18]:
#Predicting 50% probability that value is 1
#Actual label is 1
compute_log_loss(0.5, 1)

0.6931471805599453

## Possible Features to Engineer?

1. A count for number of exclamation points in a narrative - could be an indicator of how important a claim can be

## Splitting Data into Training and Test Sets

As noted above the False label only appears in about 3% of our data.  We need to make sure the training and test splits include these labels.  We can use StratifiedShuffleSplit from SciKitLearn.

## Preprocessing Data
We did some preprocessing already on the data.  Now, we will do further preprocessing to get our data for machine learning.  For this project, use a bag of words representation for NLP.

In [4]:
#Spacy tokenizer
spacy.load('en_core_web_sm')
lemmatizer = spacy.lang.en.English()

In [5]:
#Used for testing output of vectorizer
def wm2df(wm, feat_names):
    #create a dataframe from a word matrix
    # create an index for each row
    doc_names = ['Doc{:d}'.format(idx) for idx, _ in enumerate(wm)]
    df = pd.DataFrame(data=wm.toarray(), index=doc_names,
                      columns=feat_names)
    return(df)

In [6]:
#Customizing Vectorizers
def my_tokenizer(doc):
    tokens = lemmatizer(doc)
    return([token.lemma_ for token in tokens])

custom_vec = CountVectorizer(tokenizer=my_tokenizer,
                             ngram_range=(1,2),
                             stop_words='english')

cwm = custom_vec.fit_transform(['He ran through the Fields and surprisingly high speeds Petro vend'])
tokens = custom_vec.get_feature_names()
wm2df(cwm, tokens)

,field,field surprisingly,high,high speed,petro,petro vend,run,run field,speed,speed petro,surprisingly,surprisingly high,vend
Doc0,1,1,1,1,1,1,1,1,1,1,1,1,1


In [7]:
#labels we want to predict
labels = df['timely_response'].values

In [8]:
#drop the label for the Data so we can pull out only the features
df = df.drop('timely_response', axis=1)

In [9]:
#convert all features but complaint narrative to dummy variables
cats = ['product', 'sub_product', 'issue', 'sub_issue', 'company', 'submitted_via']

#df = pd.get_dummies(df, columns=cats)

In [10]:
#Use Function transformer to create steps to vectorize narrative data
def get_narr(df):
    return df['consumer_complaint_narrative']

#grabs the complaint narrative data for use in pipe line
get_narr_pl = FunctionTransformer(get_narr, validate=False)

def get_cats(df):
    return df[['product', 'sub_product', 'issue', 'sub_issue', 'company', 'submitted_via']]

get_cats_pl = FunctionTransformer(get_cats, validate=False)

## Train and Test Sets

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df, labels, test_size=.25, random_state=42)

## Create a Pipline

In [12]:
pl = Pipeline([
             ('union', FeatureUnion(
                  transformer_list = [
                      ('cat_features', Pipeline([
                          ('selector', get_cats_pl),
                          ('one_hot_encode', OneHotEncoder(handle_unknown='ignore'))
                      ])),
                      ('narr_feature', Pipeline([
                          ('selector', get_narr_pl),
                          ('vecorizer', custom_vec)
                      ]))
                  ])),
             ('clf', RandomForestClassifier())
               ])

In [ ]:
pl.fit(X_train, y_train)

In [159]:
proba = pl.predict_proba(X_test)

In [162]:
log_loss(y_test[:100], proba)

0.36420188638053297

In [163]:
pl = Pipeline([
             ('union', FeatureUnion(
                  transformer_list = [
                      ('cat_features', Pipeline([
                          ('selector', get_cats_pl),
                          ('one_hot_encode', OneHotEncoder(handle_unknown='ignore'))
                      ])),
                      ('narr_feature', Pipeline([
                          ('selector', get_narr_pl),
                          ('vecorizer', custom_vec)
                      ]))
                  ])),
             ('clf', LogisticRegression())
               ])

In [165]:
pl.fit(X_train[:100], y_train[:100])

Pipeline(memory=None,
         steps=[('union',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('cat_features',
                                                 Pipeline(memory=None,
                                                          steps=[('selector',
                                                                  FunctionTransformer(accept_sparse=False,
                                                                                      check_inverse=True,
                                                                                      func=<function get_cats at 0x00000213E1FDF048>,
                                                                                      inv_kw_args=None,
                                                                                      inverse_func=None,
                                                                                      kw_args=None,
                                       

In [167]:
proba = pl.predict_proba(X_test[:100])
log_loss(y_test[:100], proba)

0.06264046451248427